## Plot profiles from HeFESTO & Perplex outputs

In this cookbook, the profile outputs from HeFESTO is plotted. THe outpus must be generated with a single T / S and depth-varing pressures.

In [ ]:
import os, sys
import numpy as np
from shutil import rmtree, copy
from matplotlib import pyplot as plt
from matplotlib import gridspec, cm

# directory to the aspect Lab
ASPECT_LAB_DIR = os.environ['ASPECT_LAB_DIR']
RESULT_DIR = os.path.join(ASPECT_LAB_DIR, 'results')

sys.path.append(os.path.join(ASPECT_LAB_DIR))
# import shilofue.PlotDepthAverage as PDAver
# import shilofue.FlowLaws as FlowLaws
# import shilofue.ParsePrm as ParsePrm
# import shilofue.ThermalModel as TModel
# import shilofue.Rheology as Rheology
import shilofue.PostHefesto as PostHefesto

# import utilities in subdirectiory
sys.path.append(os.path.join(ASPECT_LAB_DIR, 'utilities', "python_scripts"))

HaMaGeoLib_DIR = "/home/lochy/ASPECT_PROJECT/HaMaGeoLib/hamageolib"

if HaMaGeoLib_DIR not in sys.path:
    sys.path.append(HaMaGeoLib_DIR)

import utils.plot_helper as plot_helper # plotting utilities

import Utilities

#### Plot profile I got from Gabe Epstein

In the next block, I plot the output of Garnet fraction of a NMORB composition from PerPle_X.
Gabe sent me the files in 11-2023.
I use the pcolormesh to color-mesh the garnet fraction and then plot a couple coutours on top.

In [ ]:
plt.style.use('publication_lhy11009')

perplex_path = "/home/lochy/Documents/papers/documented_files/TwoDSubduction/morb_green/morb_green_finev3_7.tab"
assert(os.path.isfile(perplex_path))

LookupTable = PostHefesto.LOOKUP_TABLE()
LookupTable.ReadPerplex(perplex_path, header_rows=13)
LookupTable.Update()
print(LookupTable.AllFields())
LookupTable.fix_field_nan_value("Gt_HGP", 0.0)
LookupTable.fix_field_nan_value("Omph_GHP", 0.0)
Ts, Ps, Gt_HGPs = LookupTable.export_field_mesh("Gt_HGP")
Ts, Ps, Omph_HGPs = LookupTable.export_field_mesh("Omph_GHP")

In [ ]:
from matplotlib import rcdefaults
from matplotlib.ticker import MultipleLocator
from matplotlib import gridspec
from cmcrameri import cm as ccm

# Retrieve the default color cycle
default_colors = [color['color'] for color in plt.rcParams['axes.prop_cycle']]

# Example usage
# Rule of thumbs:
# 1. Set the limit to something like 5.0, 10.0 or 50.0, 100.0 
# 2. Set five major ticks for each axis
scaling_factor = 1.0  # scale factor of plot
font_scaling_multiplier = 1.5 # extra scaling multiplier for font
legend_font_scaling_multiplier = 0.5
line_width_scaling_multiplier = 2.0 # extra scaling multiplier for lines

T_lim = (0.0, 1400.0) # T (C)
T_level = 50  # number of levels in contourf plot
T_tick_interval = 500.0  # tick interval along v

T_lim1 = (0.0, 800.0) # T (C), smaller scale
T_tick_interval1 = 100.0  # tick interval along x
P_lim1 = (1.0, 4.0) # P (Gpa)

P_lim = (0.0, 8.0) # P (Gpa)
P_level = 50  # number of levels in contourf plot
P_tick_interval = 1.0  # tick interval along v

n_minor_ticks = 4  # number of minor ticks between two major ones

# scale the matplotlib params
plot_helper.scale_matplotlib_params(scaling_factor, font_scaling_multiplier=font_scaling_multiplier,\
                        legend_font_scaling_multiplier=legend_font_scaling_multiplier,
                        line_width_scaling_multiplier=line_width_scaling_multiplier)

# Update font settings for compatibility with publishing tools like Illustrator.
plt.rcParams.update({
    'font.family': 'Times New Roman',
    'pdf.fonttype': 42,
    'ps.fonttype': 42
})

fig = plt.figure(tight_layout=True, figsize=(10.0, 8.5))
gs = gridspec.GridSpec(2, 2)

# garnet volume fraction
ax = fig.add_subplot(gs[0, 0])
h = ax.pcolormesh(Ts-273.15, Ps/1e4, Gt_HGPs, vmin=0.0, vmax=70.0)
ax.contour(Ts-273.15, Ps/1e4, Gt_HGPs, (10, 20, 40), cmap="Greys", linestyles="dashed")
fig.colorbar(h, ax=ax, label="Garnet (vol%)")

ax.set_xlim(T_lim)
ax.set_ylim(P_lim)

ax.set_xlabel(r"T ($^\circ C$)")
ax.set_ylabel("P (GPa)")
fig.tight_layout()

ax.xaxis.set_major_locator(MultipleLocator(T_tick_interval))
ax.xaxis.set_minor_locator(MultipleLocator(T_tick_interval/(n_minor_ticks+1)))
ax.yaxis.set_major_locator(MultipleLocator(P_tick_interval))
ax.yaxis.set_minor_locator(MultipleLocator(P_tick_interval/(n_minor_ticks+1)))

ax.invert_yaxis()

for spine in ax.spines.values():
    # Adjust spine thickness for this plot
    spine.set_linewidth(0.5 * scaling_factor * line_width_scaling_multiplier)

# clinopyroxene volume fraction
ax = fig.add_subplot(gs[0, 1])
h = ax.pcolormesh(Ts-273.15, Ps/1e4, Omph_HGPs, vmin=0.0, vmax=70.0)
ax.contour(Ts-273.15, Ps/1e4, Omph_HGPs, (10, 20, 40), cmap="Greys", linestyles="dashed")
fig.colorbar(h, ax=ax, label="Clinopyroxene (vol%)")

ax.set_xlim(T_lim)
ax.set_ylim(P_lim)

ax.set_xlabel(r"T ($^\circ C$)")
ax.set_ylabel("P (GPa)")
fig.tight_layout()

ax.xaxis.set_major_locator(MultipleLocator(T_tick_interval))
ax.xaxis.set_minor_locator(MultipleLocator(T_tick_interval/(n_minor_ticks+1)))
ax.yaxis.set_major_locator(MultipleLocator(P_tick_interval))
ax.yaxis.set_minor_locator(MultipleLocator(P_tick_interval/(n_minor_ticks+1)))

ax.invert_yaxis()

for spine in ax.spines.values():
    # Adjust spine thickness for this plot
    spine.set_linewidth(0.5 * scaling_factor * line_width_scaling_multiplier)

# garnet + clinopyroxene volume fraction
ax = fig.add_subplot(gs[1, 0])
h = ax.pcolormesh(Ts-273.15, Ps/1e4, Gt_HGPs + Omph_HGPs, vmin=0.0, vmax=70.0)
ax.contour(Ts-273.15, Ps/1e4, Gt_HGPs + Omph_HGPs, (10, 20, 40), cmap="Greys", linestyles="dashed")
fig.colorbar(h, ax=ax, label="Garnet + Clinopyroxene (vol%)")

ax.set_xlim(T_lim)
ax.set_ylim(P_lim)

ax.set_xlabel(r"T ($^\circ C$)")
ax.set_ylabel("P (GPa)")
fig.tight_layout()

ax.xaxis.set_major_locator(MultipleLocator(T_tick_interval))
ax.xaxis.set_minor_locator(MultipleLocator(T_tick_interval/(n_minor_ticks+1)))
ax.yaxis.set_major_locator(MultipleLocator(P_tick_interval))
ax.yaxis.set_minor_locator(MultipleLocator(P_tick_interval/(n_minor_ticks+1)))

ax.invert_yaxis()

for spine in ax.spines.values():
    # Adjust spine thickness for this plot
    spine.set_linewidth(0.5 * scaling_factor * line_width_scaling_multiplier)

fig_path = os.path.join(RESULT_DIR, "Perple_X_morb_garnet.pdf")
fig.savefig(fig_path)
print("Figure saved: %s" % fig_path)

# Reset rcParams to defaults
rcdefaults()

In [ ]:
print(LookupTable.header)

#### Table to export to ASPECT

Pay attention to the first dimension (T) and the second dimension (P)

These values has to be passed to the function "Process" correctly

In [ ]:
Gt_HGP_array = LookupTable.export_field("Gt_HGP")
Omph_HGP_array = LookupTable.export_field("Omph_GHP")

EC_array = Gt_HGP_array + Omph_HGP_array
EC_percent_array = EC_array / 100.0
new_name = "eclogite"
oheader = "vol_fraction_eclogite"
LookupTable.CreateNew(EC_percent_array.reshape([EC_percent_array.size, 1]), new_name, oheader)

field_names = ["Temperature", 'Pressure', 'Density', new_name]
# o_path = os.path.join(RESULT_DIR, "perplex_morb_test.txt")
o_path = os.path.join(RESULT_DIR, "perplex_morb_test_small.txt")
LookupTable.Process(field_names, o_path, first_dimension="Temperature", second_dimension="Pressure", interval1=10, interval2=10)